# Gradient Checking

In [1]:
# Packages
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 1-dimensional gradient checking

Implement "forward propagation" and "backward propagation" for this simple function. I.e., compute both  𝐽(.)  ("forward propagation") and its derivative with respect to  𝜃  ("backward propagation"), in two separate functions.

In [2]:
def forward_prop(x, theta):
    
    J = theta*x
    
    return J

In [4]:
x, theta = 2, 4
J = forward_prop(x, theta)
print ("J = " + str(J))

J = 8


In [5]:
def backward_prop(x, theta):
    
    dtheta = x
    
    return dtheta

In [7]:
x, theta = 2, 4
dtheta = backward_prop(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


To show that the `backward_prop()` function is correctly computing the gradient $\frac{\partial J}{\partial \theta}$, let's implement gradient checking.

- First compute "gradapprox" and a small value of $\varepsilon$. Here are the Steps to follow:
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- Then compute the gradient using backward propagation, and store the result in a variable "grad"
- Finally, compute the relative difference between "gradapprox" and the "grad" using the following formula:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} $$

- If this difference is small (say less than $10^{-7}$), you can be quite confident that you have computed your gradient correctly. Otherwise, there may be a mistake in the gradient computation. 


In [19]:
def gradient_check(x, theta, epsilon = 1e-7):
    
    theta_plus = theta + epsilon
    theta_minus = theta - epsilon
    J_plus = forward_prop(x, theta_plus)
    J_minus = forward_prop(x, theta_minus)
    gradapprox = (J_plus - J_minus)/(2 * epsilon)
    
    grad = backward_prop(x, theta)
    
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    
    difference = numerator/denominator
    
    if difference < epsilon:
        print("Gradient is correct")
    else:
        print("Gradient is wrong")
        
    
    return difference

In [20]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

Gradient is correct
difference = 2.919335883291695e-10


## N-dimensional gradient checking

In [61]:
def forward_prop(X, Y, parameters):
    
    m = X.shape[1]
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [62]:
def backward_prop(X, Y, cache):
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [65]:
def gradient_checking(parameters, gradients, X, Y, epsilon = 1e-7):
    
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters,1))
    J_minus = np.zeros((num_parameters,1))
    gradapprox = np.zeros((num_parameters,1))
    
    for i in range(num_parameters):
        
        thetaplus = np.copy(parameters_values)
        thetaplus[i][0] = thetaplus[i][0] + epsilon
        J_plus[i], _ = forward_prop(X, Y, vector_to_dictionary(thetaplus))
        
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] = thetaminus[i][0] - epsilon
        J_minus[i], _ = forward_prop(X, Y, vector_to_dictionary(thetaminus))
        
        gradapprox[i] = (J_plus[i] - J_minus[i])/(2*epsilon)
        
        
    numerator = np.linalg.norm(grad-gradapprox)                                           # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                          # Step 2'
    difference = numerator/denominator                                                        # Step 3'
    ### END CODE HERE ###

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [66]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_prop(X, Y, parameters)
gradients = backward_prop(X, Y, cache)
difference = gradient_checking(parameters, gradients, X, Y)

Your backward propagation works perfectly fine! difference = 1.1890417878779318e-07
